<a href="https://colab.research.google.com/github/2303A51097/AIML-2025/blob/main/lab3_AIML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#part-1 implement the AI game Strategy


##Install the Python Libraries required for Game Strategy.

In [9]:
from collections import namedtuple, Counter, defaultdict
import random
import math
import functools

# Define the cache using functools.lru_cache
cache = functools.lru_cache(10**6)

#GAME CLASS


In [10]:
class Game:
    def actions(self, state):
        """Return a collection of the allowable moves from this state."""
        raise NotImplementedError

    def result(self, state, move):
        """Return the state that results from making a move from a state."""
        raise NotImplementedError

    def is_terminal(self, state):
        """Return True if this is a final state for the game."""
        return not self.actions(state)

    def utility(self, state, player):
        """Return the value of this final state to player."""
        raise NotImplementedError

##Implement a Player Game using the Game Class Constructor


In [11]:
def play_game(game, strategies: dict, verbose=False):
    """
    Play a turn-taking game. strategies is a dictionary where the keys are player names,
    and the values are functions that take the current state and the game as arguments,
    returning the player's move.
    """
    state = game.initial
    while not game.is_terminal(state):
        player = state.to_move
        move = strategies[player](state, game)
        state = game.result(state, move)
        if verbose:
            print('Player', player, 'move:', move)
            print(state)
    return state

#Part 2 -Implement the Game Strategy Algorithms.

# Implement the MiniMax Search Algorithm

In [12]:
import math

def minimax_search(game, state):
    # Search game tree to determine the best move; return (value, move) pair.
    player = state.to_move

    def max_value(state):
        if game.is_terminal(state):
            return game.utility(state, player), None

        v, move = -math.inf, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a))
            if v2 > v:
                v, move = v2, a
        return v, move

    def min_value(state):
        if game.is_terminal(state):
            return game.utility(state, player), None

        v, move = math.inf, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a))
            if v2 < v:
                v, move = v2, a
        return v, move

    return max_value(state)


#2. Implement the Alpha-Beta Search Algorithm

In [13]:
import math

def alphabeta_search(game, state):
    # Search game tree to determine the best action; use alpha-beta pruning.
    # Search all the way to the leaves.
    player = state.to_move

    def max_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None

        v, move = -math.inf, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a), alpha, beta)
            if v2 > v:
                v, move = v2, a
            alpha = max(alpha, v)
            if v >= beta:
                return v, move
        return v, move

    def min_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None

        v, move = math.inf, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a), alpha, beta)
            if v2 < v:
                v, move = v2, a
            beta = min(beta, v)
            if v <= alpha:
                return v, move
        return v, move

    return max_value(state, -math.inf, math.inf)


#Part 3 Implement the Game Strategy using TicTocToe.

##Implement TicToCToe game using init , actions, result, is terminal, utilitydisplay constructors

In [6]:
class TicTacToe(Game):
    """Play TicTacToe on a height x width board, needing k in a row to win.
       X plays first against O."""

    def __init__(self, height=3, width=3, k=3):
        self.k = k  # k in a row to win
        self.height = height
        self.width = width
        self.squares = [(x, y) for x in range(width) for y in range(height)]
        self.initial = Board(height=height, width=width, to_move='X', utility=0)

    def actions(self, board):
        """Legal moves are any square not yet taken."""
        return set(self.squares) - set(board)

    def result(self, board, square):
        """Place a marker for the current player on the square."""
        player = board.to_move
        next_player = 'O' if player == 'X' else 'X'
        board = board.new(square=square, player=player, to_move=next_player)
        win = self.k_in_row(board, player, square, self.k)
        board.utility = (0 if not win else +1 if player == 'X' else -1)
        return board

    def utility(self, board, player):
        """Return the value to player; 1 for win, -1 for loss, 0 otherwise."""
        return board.utility if player == 'X' else -board.utility

    def is_terminal(self, board):
        """A board is a terminal state if it is won or there are no empty squares."""
        return board.utility != 0 or len(self.squares) == len(board)

    def display(self, board):
        """Print the board."""
        print(board)

    def k_in_row(self, board, player, square, k):
        """Check if the player has k pieces in a line through square."""
        def in_row(x, y, dx, dy):
            """Count consecutive pieces of the player in the direction (dx, dy)."""
            if board.get((x, y)) != player:
                return 0
            return 1 + in_row(x + dx, y + dy, dx, dy)

        return any(
            in_row(square[0], square[1], dx, dy) + in_row(square[0], square[1], -dx, -dy) - 1 >= k
            for dx, dy in [(0, 1), (1, 0), (1, 1), (1, -1)]
        )


# 2. Implement a Game Board using defaultdict using init , new, missing , hash ,repr



In [7]:
from collections import defaultdict

class Board(defaultdict):
    """A board has the player to move, a cached utility value,
       and a dict of (x, y): player entries, where player is 'X' or 'O'.
       empty squares are denoted by '.', and off-board positions by '#'.
    """

    empty = '.'
    off = '#'

    def __init__(self, width=8, height=8, to_move=None, **kwds):
        super().__init__(self.missing)
        self.width = width
        self.height = height
        self.to_move = to_move
        self.update(**kwds)

    def new(self, changes: dict, **kwds) -> 'Board':
        """Given a dict of (x, y): contents changes, return a new Board with the changes."""
        board = Board(width=self.width, height=self.height, to_move=self.to_move, **kwds)
        board.update(self)
        board.update(changes)
        return board

    def missing(self, loc):
        """Return the value for a location that is not on the board."""
        x, y = loc
        if 0 <= x < self.width and 0 <= y < self.height:
            return self.empty
        else:
            return self.off

    def __hash__(self):
        """Hash the board based on its contents and the player to move."""
        return hash(tuple(sorted(self.items()))) + hash(self.to_move)

    def __repr__(self):
        """String representation of the board."""
        def row(y):
            return ''.join(self[x, y] for x in range(self.width))
        return '\n'.join(map(row, range(self.height))) + '\n'


# 3. Implement random player(game,state) and player(search algorithm)

In [8]:

.import random

def random_player(game, state):
    """Return a random move from the available actions."""
    return random.choice(list(game.actions(state)))

def player(search_algorithm):
    """Return a game player function that uses the specified search algorithm."""
    return lambda game, state: search_algorithm(game, state)[1]


## Evaluate the AI Game Strategy using TicTocToe.

# Implement GameStrategy using